In [44]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from tqdm import tqdm
import random
import seaborn as sns
import matplotlib.pyplot as plt
from geopy.geocoders import Nominatim
import warnings
warnings.filterwarnings('ignore')

import os
import plotly.express as px

import xgboost as xg
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

In [3]:

mydict= {'ID': list(),
 'Delivery_person_ID': list(),
 'Delivery_person_Age': list(),
 'Delivery_person_Ratings': list(),
 'Restaurant_latitude': list(),
 'Restaurant_longitude': list(),
 'Delivery_location_latitude': list(),
 'Delivery_location_longitude': list(),
 'Order_Date': list(),
 'Time_Orderd': list(),
 'Time_Order_picked': list(),
 'Weather': list(),
 'Road_traffic_density': list(),
 'Vehicle_condition': list(),
 'Type_of_order': list(),
 'Type_of_vehicle': list(),
 'multiple_deliveries': list(),
 'Festival': list(),
 'City': list(),
 'Time_taken': list(),
 'Name:': list()}

In [4]:
mydict_1= {'ID': list(),
 'Delivery_person_ID': list(),
 'Delivery_person_Age': list(),
 'Delivery_person_Ratings': list(),
 'Restaurant_latitude': list(),
 'Restaurant_longitude': list(),
 'Delivery_location_latitude': list(),
 'Delivery_location_longitude': list(),
 'Order_Date': list(),
 'Time_Orderd': list(),
 'Time_Order_picked': list(),
 'Weather': list(),
 'Road_traffic_density': list(),
 'Vehicle_condition': list(),
 'Type_of_order': list(),
 'Type_of_vehicle': list(),
 'multiple_deliveries': list(),
 'Festival': list(),
 'City': list(),
 'Name:': list()}

In [5]:
def get_train_data(path):
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            with open(os.path.join(dirname, filename),"r") as file:
                f=file.readlines()
                for i in range(len(f)):
                    k=list(f[i].split())
                    if k[0]== "Time_taken":
                        mydict[k[0]].append(k[2])
                    else:
                        mydict[k[0]].append(k[1])
    data = pd.DataFrame(mydict)
    data = data.drop(["Name:"],axis=1)
    return data

def get_test_data(path):
    for dirname, _, filenames in os.walk(path):
        for filename in filenames:
            with open(os.path.join(dirname, filename),"r") as file:
                f=file.readlines()
                for i in range(len(f)):
                    k=list(f[i].split())
                    if k[0]== "Time_taken":
                        mydict_1[k[0]].append(k[2])
                    else:
                        mydict_1[k[0]].append(k[1])
    data = pd.DataFrame(mydict_1)
    data = data.drop(["Name:"],axis=1)
    return data

train = get_train_data("/Users/vikram/Python/Food-Order-ETA-POC/dataset/train")
test = get_test_data("/Users/vikram/Python/Food-Order-ETA-POC/dataset/test")

In [6]:
train.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken
0,0x6b1a,JAPRES05DEL01,28.000000,4.900000,26.905287,75.794592,26.915287,75.804592,03-04-2022,8:15,8:20,conditions,Low,2,Snack,electric_scooter,0.000000,No,Urban,15.000000
1,0xb46,PUNERES20DEL02,34.000000,3.500000,18.592718,73.773572,18.672718,73.853572,20-03-2022,17:25,17:35,conditions,Medium,2,Buffet,scooter,2.000000,No,Metropolitian,37.000000
2,0x6adc,BANGRES19DEL02,34.000000,4.900000,12.914264,77.678400,12.964264,77.728400,17-03-2022,21:20,21:25,conditions,Jam,1,Snack,motorcycle,1.000000,No,Metropolitian,23.000000
3,0x4477,COIMBRES12DEL03,37.000000,4.800000,11.000762,76.981876,11.060762,77.041876,28-03-2022,17:40,17:55,conditions,Medium,0,Meal,motorcycle,1.000000,No,Metropolitian,29.000000
4,0x48ee,JAPRES19DEL01,38.000000,4.700000,26.913726,75.752820,26.953726,75.792821,01-04-2022,12:45,12:60,conditions,High,1,Meal,motorcycle,1.000000,No,Metropolitian,17.000000


In [7]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ID                           45593 non-null  object
 1   Delivery_person_ID           45593 non-null  object
 2   Delivery_person_Age          45593 non-null  object
 3   Delivery_person_Ratings      45593 non-null  object
 4   Restaurant_latitude          45593 non-null  object
 5   Restaurant_longitude         45593 non-null  object
 6   Delivery_location_latitude   45593 non-null  object
 7   Delivery_location_longitude  45593 non-null  object
 8   Order_Date                   45593 non-null  object
 9   Time_Orderd                  45593 non-null  object
 10  Time_Order_picked            45593 non-null  object
 11  Weather                      45593 non-null  object
 12  Road_traffic_density         45593 non-null  object
 13  Vehicle_condition            45

In [8]:
test.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City
0,0x9092,JAPRES18DEL02,27.000000,4.500000,26.913987,75.752891,27.043987,75.882891,20-03-2022,23:40,23:45,conditions,Low,0,Snack,motorcycle,1.000000,No,Urban
1,0x3999,COIMBRES02DEL03,24.000000,4.600000,11.022477,76.995667,11.052477,77.025667,03-04-2022,17:40,17:55,conditions,Medium,1,Drinks,motorcycle,1.000000,No,Urban
2,0x5dc8,COIMBRES13DEL01,36.000000,3.600000,11.003669,76.976494,11.073669,77.046494,16-03-2022,17:25,17:35,conditions,Medium,1,Meal,motorcycle,1.000000,No,Metropolitian
3,0xaa48,INDORES010DEL03,20.000000,4.400000,22.750040,75.902847,22.840040,75.992847,31-03-2022,20:50,21:05,conditions,Jam,1,Drinks,motorcycle,1.000000,No,Metropolitian
4,0x46eb,SURRES03DEL01,35.000000,4.900000,21.186884,72.793616,21.196884,72.803616,05-03-2022,10:10,10:20,conditions,Low,1,Drinks,scooter,1.000000,No,Metropolitian


In [9]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11399 entries, 0 to 11398
Data columns (total 19 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   ID                           11399 non-null  object
 1   Delivery_person_ID           11399 non-null  object
 2   Delivery_person_Age          11399 non-null  object
 3   Delivery_person_Ratings      11399 non-null  object
 4   Restaurant_latitude          11399 non-null  object
 5   Restaurant_longitude         11399 non-null  object
 6   Delivery_location_latitude   11399 non-null  object
 7   Delivery_location_longitude  11399 non-null  object
 8   Order_Date                   11399 non-null  object
 9   Time_Orderd                  11399 non-null  object
 10  Time_Order_picked            11399 non-null  object
 11  Weather                      11399 non-null  object
 12  Road_traffic_density         11399 non-null  object
 13  Vehicle_condition            11

In [10]:
print(train.shape)
print(test.shape)

(45593, 20)
(11399, 19)


In [12]:
def get_NaN(data):
    for i in data.columns:
        counter=0
        for j in range(data.shape[0]):
            if data[i].iloc[j]=="NaN":
                counter+=1
            else:
                pass
        print(i,counter)
get_NaN(train)


ID 0
Delivery_person_ID 0
Delivery_person_Age 1854
Delivery_person_Ratings 1908
Restaurant_latitude 0
Restaurant_longitude 0
Delivery_location_latitude 0
Delivery_location_longitude 0
Order_Date 0
Time_Orderd 1731
Time_Order_picked 0
Weather 0
Road_traffic_density 601
Vehicle_condition 0
Type_of_order 0
Type_of_vehicle 0
multiple_deliveries 993
Festival 228
City 1200
Time_taken 0


In [13]:
get_NaN(test)

ID 0
Delivery_person_ID 0
Delivery_person_Age 491
Delivery_person_Ratings 507
Restaurant_latitude 0
Restaurant_longitude 0
Delivery_location_latitude 0
Delivery_location_longitude 0
Order_Date 0
Time_Orderd 444
Time_Order_picked 0
Weather 0
Road_traffic_density 154
Vehicle_condition 0
Type_of_order 0
Type_of_vehicle 0
multiple_deliveries 238
Festival 65
City 324


In [14]:
def check(data):
    month,year=[],[]
    for i in range(data.shape[0]):
        d=data["Order_Date"].iloc[i].split("-")
        month.append(d[1])
        year.append(d[2])
    print("Months",set(month))
    print("Year", set(year))
print("Train : ")
check(train)
print("Test : ")
check(test)

Train : 
Months {'02', '03', '04'}
Year {'2022'}
Test : 
Months {'02', '03', '04'}
Year {'2022'}


In [15]:
train["Festival"].value_counts()

No     44469
Yes      896
NaN      228
Name: Festival, dtype: int64

In [16]:
test["Festival"].value_counts()

No     11131
Yes      203
NaN       65
Name: Festival, dtype: int64

In [17]:
l=list(train[train.Festival=="Yes"].Order_Date)
l.extend(list(test[test.Festival=="Yes"].Order_Date))
holidays=list(set(l))
print("all the Festivals date")
holidays


all the Festivals date


['17-02-2022',
 '20-03-2022',
 '25-03-2022',
 '28-03-2022',
 '30-03-2022',
 '06-03-2022',
 '04-03-2022',
 '02-03-2022',
 '16-03-2022',
 '01-04-2022',
 '18-02-2022',
 '17-03-2022',
 '19-03-2022',
 '15-03-2022',
 '09-03-2022',
 '14-03-2022',
 '06-04-2022',
 '12-02-2022',
 '16-02-2022',
 '14-02-2022',
 '10-03-2022',
 '03-03-2022',
 '07-03-2022',
 '05-03-2022',
 '01-03-2022',
 '08-03-2022',
 '11-02-2022',
 '05-04-2022',
 '23-03-2022',
 '03-04-2022',
 '12-03-2022',
 '13-02-2022',
 '29-03-2022',
 '21-03-2022',
 '18-03-2022',
 '02-04-2022',
 '24-03-2022',
 '13-03-2022',
 '04-04-2022',
 '31-03-2022',
 '15-02-2022',
 '26-03-2022',
 '11-03-2022',
 '27-03-2022']

In [18]:
def fill_NaN_festival(data):
    for i in range(data.shape[0]):
        if data["Festival"].iloc[i]=="NaN":
            if data["Order_Date"].iloc[i] in holidays:
                data["Festival"].iloc[i]="Yes"
            else:
                data["Festival"].iloc[i]="No"
        else:
            pass
    data["Festival"]=data["Festival"].astype('category')
fill_NaN_festival(train)
fill_NaN_festival(test)


In [19]:
train["Festival"].value_counts()


No     44469
Yes     1124
Name: Festival, dtype: int64

In [20]:
test["Festival"].value_counts()

No     11131
Yes      268
Name: Festival, dtype: int64

In [21]:
def clean_multiple_deliveries(data):
    for i in tqdm(range(data.shape[0])):
        if data.multiple_deliveries.iloc[i] != "NaN":
            data.multiple_deliveries.iloc[i] = int(str(data.multiple_deliveries.iloc[i])[0])
            pass
        else:
            # replace NaN with 1
            data.multiple_deliveries.iloc[i] = 1
    data["multiple_deliveries"]=data["multiple_deliveries"].astype('int64')
clean_multiple_deliveries(train)
clean_multiple_deliveries(test)

100%|██████████| 11399/11399 [00:01<00:00, 10553.33it/s]


In [23]:
def clean_delivery_person_age(data):
    for i in tqdm(range(data.shape[0])):
        if data.Delivery_person_Age.iloc[i] != "NaN":
            data.Delivery_person_Age.iloc[i] = int(str(data.Delivery_person_Age.iloc[i])[:2])
        else:
            # replace NaN with median
            data.Delivery_person_Age.iloc[i] = data.Delivery_person_Age.median()
            # replace NaN with mode
            #data.Delivery_person_Age.iloc[i] = data.Delivery_person_Age.mode()
#train["Delivery_person_Age"]=pd.to_numeric(train["Delivery_person_Age"], downcast='float')
clean_delivery_person_age(train)
clean_delivery_person_age(test)
train["Delivery_person_Age"]=train["Delivery_person_Age"].astype('int64')
test["Delivery_person_Age"]=test["Delivery_person_Age"].astype('int64')


100%|██████████| 11399/11399 [00:00<00:00, 15285.39it/s]


In [24]:
def clean_Delivery_person_Ratings(data):
    mod = data.Delivery_person_Ratings.mode()
    for i in tqdm(range(data.shape[0])):
        #print(i)
        if data.Delivery_person_Ratings.iloc[i] != "NaN":
            data.Delivery_person_Ratings.iloc[i] = float(str(data.Delivery_person_Ratings.iloc[i])[:3])
        else:
            # replace NaN with median
            data.Delivery_person_Ratings.iloc[i] = data.Delivery_person_Ratings.median()
            
            #replace NaN with mode
            # data.Delivery_person_Ratings.iloc[i] = data.Delivery_person_Ratings.mode()
    data["Delivery_person_Ratings"]=pd.to_numeric(data["Delivery_person_Ratings"], downcast='float')
            
clean_Delivery_person_Ratings(train)
clean_Delivery_person_Ratings(test)

100%|██████████| 11399/11399 [00:01<00:00, 8826.49it/s]


In [25]:
train[(train.City=="NaN") & (train.Road_traffic_density=="NaN")]

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken
833,0x50b,MUMRES03DEL01,30,4.7,-19.223840,-72.841347,19.263840,72.881347,05-03-2022,NaN,14:05,conditions,NaN,3,Meal,electric_scooter,0,No,NaN,34.000000
3386,0x590,RANCHIRES05DEL03,30,4.7,-23.355164,85.324097,23.385164,85.354097,05-04-2022,NaN,23:60,conditions,NaN,3,Buffet,scooter,0,No,NaN,26.000000
6946,0x6e5,INDORES11DEL02,30,4.7,22.751234,75.889490,22.771234,75.909490,21-03-2022,NaN,10:30,conditions,NaN,3,Buffet,scooter,0,No,NaN,15.000000
7375,0x44b,BANGRES18DEL02,30,4.7,12.913041,77.683237,13.043041,77.813237,31-03-2022,NaN,23:10,conditions,NaN,3,Buffet,electric_scooter,1,No,NaN,28.000000
7534,0x4f6,COIMBRES14DEL01,30,4.7,-11.003681,-76.975525,11.073681,77.045525,02-04-2022,NaN,18:35,conditions,NaN,3,Meal,bicycle,0,No,NaN,20.000000
14019,0x496,CHENRES07DEL01,30,4.7,-13.081878,-80.248519,13.091878,80.258519,17-03-2022,NaN,9:50,conditions,NaN,3,Snack,motorcycle,1,No,NaN,19.000000
19404,0x1067,VADRES03DEL03,30,4.7,22.32,73.17,22.41,73.26,25-03-2022,NaN,23:55,conditions,NaN,3,Snack,electric_scooter,0,No,NaN,10.00
19553,0x3f4,COIMBRES20DEL02,50,6.0,-10.994136,-76.963303,11.074136,77.043303,06-03-2022,NaN,17:20,conditions,NaN,3,Drinks,electric_scooter,0,No,NaN,17.000000
29283,0x605,MYSRES20DEL02,30,4.7,12.337978,76.616792,12.387978,76.666792,07-03-2022,NaN,17:50,conditions,NaN,1,Meal,scooter,1,No,NaN,16.000000
30158,0xbf08,LUDHRES17DEL01,30,4.7,-30.893384,-75.821202,30.903384,75.831202,11-02-2022,NaN,9:35,conditions,NaN,3,Drinks,bicycle,0,No,NaN,15.000000


In [27]:
temp={ "Metropolitian" : ['Jam', 'Low', 'Medium'] ,
        "Urban" : ['Low' , 'Jam' , 'Medium'] ,
      "Semi-Urban" : ["Jam"]
}

def clean_Road_traffic_density(data):
    for i in tqdm(range(data.shape[0])):
        if data.Road_traffic_density.iloc[i] == 'NaN':
            if data.City.iloc[i] == 'Metropolitian':
                data.Road_traffic_density.iloc[i] = random.choice(temp['Metropolitian'])
            elif data.City.iloc[i] == 'Urban':
                data.Road_traffic_density.iloc[i] = random.choice(temp['Urban'])
            elif data.City.iloc[i] == 'Semi-Urban':
                data.Road_traffic_density.iloc[i] = random.choice(temp['Semi-Urban'])
            else:
                data.Road_traffic_density.iloc[i] = random.choice(temp['Metropolitian'])
        else:
            pass
    data["Road_traffic_density"]=data["Road_traffic_density"].astype('category')
clean_Road_traffic_density(train)
clean_Road_traffic_density(test)

100%|██████████| 11399/11399 [00:00<00:00, 130982.59it/s]


In [28]:
t= test["ID"]

In [29]:
train = train.drop(["City", "Time_Orderd", "ID", "Delivery_person_ID"],axis=1)
test = test.drop(["City", "Time_Orderd", "ID", "Delivery_person_ID" ],axis=1)


In [30]:
def lat_long(data):
    """for i in tqdm(range(data.shape[0])):
        data["Restaurant_latitude"].iloc[i]= float(str(data["Restaurant_latitude"].iloc[i]))
        data["Restaurant_longitude"].iloc[i]= float(str(data["Restaurant_longitude"].iloc[i]))
        data["Delivery_location_latitude"].iloc[i]= float(str(data["Delivery_location_latitude"].iloc[i]))    
        data["Delivery_location_longitude"].iloc[i]= float(str(data["Delivery_location_longitude"].iloc[i]))
    """
    data["Restaurant_longitude"]=pd.to_numeric(data["Restaurant_longitude"], downcast='float')
    data["Delivery_location_latitude"]=pd.to_numeric(data["Delivery_location_latitude"], downcast='float')
    data["Delivery_location_longitude"]=pd.to_numeric(data["Delivery_location_longitude"], downcast='float')
    data["Restaurant_latitude"]=pd.to_numeric(data["Restaurant_latitude"], downcast='float')
lat_long(train)
lat_long(test)

In [31]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 16 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   Delivery_person_Age          45593 non-null  int64   
 1   Delivery_person_Ratings      45593 non-null  float32 
 2   Restaurant_latitude          45593 non-null  float32 
 3   Restaurant_longitude         45593 non-null  float32 
 4   Delivery_location_latitude   45593 non-null  float32 
 5   Delivery_location_longitude  45593 non-null  float32 
 6   Order_Date                   45593 non-null  object  
 7   Time_Order_picked            45593 non-null  object  
 8   Weather                      45593 non-null  object  
 9   Road_traffic_density         45593 non-null  category
 10  Vehicle_condition            45593 non-null  object  
 11  Type_of_order                45593 non-null  object  
 12  Type_of_vehicle              45593 non-null  object  
 13  m

In [32]:
train.head()

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Order_picked,Weather,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,Time_taken
0,28,4.9,26.905287,75.794594,26.915287,75.804588,03-04-2022,8:20,conditions,Low,2,Snack,electric_scooter,0,No,15.000000
1,34,3.5,18.592718,73.773575,18.672718,73.853569,20-03-2022,17:35,conditions,Medium,2,Buffet,scooter,2,No,37.000000
2,34,4.9,12.914264,77.678398,12.964264,77.728401,17-03-2022,21:25,conditions,Jam,1,Snack,motorcycle,1,No,23.000000
3,37,4.8,11.000762,76.981873,11.060762,77.041878,28-03-2022,17:55,conditions,Medium,0,Meal,motorcycle,1,No,29.000000
4,38,4.7,26.913727,75.752823,26.953726,75.792824,01-04-2022,12:60,conditions,High,1,Meal,motorcycle,1,No,17.000000


In [33]:
train=train.drop(['Order_Date'],axis=1)
test=test.drop(['Order_Date'],axis=1)


In [34]:
def config_Time_Order_picked(data):
    for i in tqdm(range(data.shape[0])):
        data.Time_Order_picked.iloc[i] = float(str(data.Time_Order_picked.iloc[i]).replace(":","."))
    data["Time_Order_picked"]=pd.to_numeric(data["Time_Order_picked"], downcast='float')
config_Time_Order_picked(train)
config_Time_Order_picked(test)

100%|██████████| 11399/11399 [00:00<00:00, 13365.61it/s]


In [35]:
def prepocess_Time_taken(data):
   
    data["Time_taken"]=pd.to_numeric(data["Time_taken"], downcast='integer')
prepocess_Time_taken(train)


In [36]:
def change_types(data):
    data["Weather"]=data["Weather"].astype('category')
    data["Vehicle_condition"]=data["Vehicle_condition"].astype('int64')
    data["Type_of_order"]=data["Type_of_order"].astype('category')
    data["Type_of_vehicle"]=data["Type_of_vehicle"].astype('category')
change_types(train)
change_types(test)

In [37]:
train.head()

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Time_Order_picked,Weather,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,Time_taken
0,28,4.9,26.905287,75.794594,26.915287,75.804588,8.200000,conditions,Low,2,Snack,electric_scooter,0,No,15
1,34,3.5,18.592718,73.773575,18.672718,73.853569,17.350000,conditions,Medium,2,Buffet,scooter,2,No,37
2,34,4.9,12.914264,77.678398,12.964264,77.728401,21.250000,conditions,Jam,1,Snack,motorcycle,1,No,23
3,37,4.8,11.000762,76.981873,11.060762,77.041878,17.549999,conditions,Medium,0,Meal,motorcycle,1,No,29
4,38,4.7,26.913727,75.752823,26.953726,75.792824,12.600000,conditions,High,1,Meal,motorcycle,1,No,17


In [38]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   Delivery_person_Age          45593 non-null  int64   
 1   Delivery_person_Ratings      45593 non-null  float32 
 2   Restaurant_latitude          45593 non-null  float32 
 3   Restaurant_longitude         45593 non-null  float32 
 4   Delivery_location_latitude   45593 non-null  float32 
 5   Delivery_location_longitude  45593 non-null  float32 
 6   Time_Order_picked            45593 non-null  float32 
 7   Weather                      45593 non-null  category
 8   Road_traffic_density         45593 non-null  category
 9   Vehicle_condition            45593 non-null  int64   
 10  Type_of_order                45593 non-null  category
 11  Type_of_vehicle              45593 non-null  category
 12  multiple_deliveries          45593 non-null  int64   
 13  F

In [39]:
train=pd.get_dummies(train)
test=pd.get_dummies(test)

In [40]:
print(train.shape)
print(test.shape)

(45593, 25)
(11399, 24)


In [41]:
X,y = train.drop(["Time_taken"],axis=1),train["Time_taken"]

In [42]:
train_X, test_X, train_y, test_y = train_test_split(X, y,
                      test_size = 0.3, random_state = 123)

In [45]:
import xgboost as xgb
#=========================================================================
# XGBoost regression: 
# Parameters: 
# n_estimators  "Number of gradient boosted trees. Equivalent to number 
#                of boosting rounds."
# learning_rate "Boosting learning rate (also known as “eta”)"
# max_depth     "Maximum depth of a tree. Increasing this value will make 
#                the model more complex and more likely to overfit." 
#=========================================================================
regressor=xgb.XGBRegressor(eval_metric=r2_score)

#=========================================================================
# exhaustively search for the optimal hyperparameters
#=========================================================================
from sklearn.model_selection import GridSearchCV
# set up our search grid
param_grid = {"max_depth":    [5, 6, 7],
              "n_estimators": [800, 600, 700],
              "learning_rate": [0.01, 0.02,0.03,0.05,0.10,0.15]}

# try out every combination of the above values
search = GridSearchCV(regressor, param_grid, cv=5).fit(train_X, train_y)

print("The best hyperparameters are ",search.best_params_)


The best hyperparameters are  {'learning_rate': 0.02, 'max_depth': 6, 'n_estimators': 800}


In [49]:
xgb_r = xg.XGBRegressor(n_estimators=800, max_depth=6, learning_rate = 0.02)

  
# Fitting the model
xgb_r.fit(train_X, train_y)
  
# Predict the model
pred = xgb_r.predict(test_X)
  
# RMSE Computation
r2= r2_score(test_y, pred)
print("r2_score : % f" %(r2))

r2_score :  0.624975


In [50]:
result= {"ID":t, "Time_taken (min)":list(xgb_r.predict(test))}
res = pd.DataFrame(result)
res["Time_taken (min)"]=pd.to_numeric(res["Time_taken (min)"], downcast='float')
for i in tqdm(range(res.shape[0])):
    res["Time_taken (min)"].iloc[i]= round(float(round(res["Time_taken (min)"].iloc[i])),1)

100%|██████████| 11399/11399 [00:01<00:00, 9923.85it/s]


In [51]:
res.head()

,ID,Time_taken (min)
0,0x9092,22.0
1,0x3999,21.0
2,0x5dc8,34.0
3,0xaa48,36.0
4,0x46eb,20.0


In [52]:
res.to_csv("result1.csv", index=False)

In [57]:
import baseten
baseten.login("3oVcy9Z8.2ICLeSJp8VXh2pfUi8fa63NQcy5uPpKd")
baseten.deploy(
    xgb_r,  # e.g: a PyTorch model MyModel
    model_name='My XGBoost model',
    #model_files=['/Users/vikram/Python/Food-Order-ETA-POC/model.py'],
)

INFO API key set.


                different interface during inference than the class
                you used to train this model. You can learn more about
                these differences at
                https://xgboost.readthedocs.io/en/stable/tutorials/saving_model.html
                


INFO 
INFO --------------------------------------------------------------------------------------------
INFO |  Autogenerating Truss for your model, find more about Truss at https://truss.baseten.co/ |
INFO --------------------------------------------------------------------------------------------
INFO You can find your auto generated Truss at  /Users/vikram/.truss/models/xgboost-4GUP7J
INFO ---------------------------------------------------------------
INFO |  Some useful commands to work with your Truss... |
INFO ---------------------------------------------------------------
INFO To load your Truss, you can either
INFO 1. Access the Truss handle via the object returned by this function via  deployed_model.truss
INFO 2. Load the Truss handle via  handle = truss.from_directory('/Users/vikram/.truss/models/xgboost-4GUP7J')
INFO To locally run your Truss before deploying, run  handle.server_predict(test_input)
INFO To add a requirement to your Truss, run  handle.add_python_requirement

BasetenDeployedModel<
  model_version_id=nwxn153
  name=My XGBoost model
>